<a href="https://colab.research.google.com/github/kk20krishna/Intelligent-Sourcing/blob/main/Intelligent_Sourcing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Intelligent Sourcing**

Created by Krishna Kumar Sankaran Kutty

# Housekeeping

In [58]:
!pip install pulp xlsxwriter

In [1]:
# Import PuLP modeler functions
from pulp import *
import pandas as pd
from io import StringIO
import numpy as np
import xlsxwriter
import matplotlib.pyplot as plt

In [10]:
rng = np.random.default_rng(seed=42)
print(rng.integers(1, 100))
print(rng.integers(1, 100))
print(rng.integers(1, 100))
print(rng.integers(1, 100))
print(rng.integers(1, 100))
print(rng.integers(1, 100))
print(rng.integers(1, 100))



9
77
65
44
43
86
9


In [60]:
rng = np.random.default_rng(seed=42)

# Input Data

## Business Strategy Prioritization

In [ ]:
import pandas as pd

def read_all_sheets(filepath):
  """Reads all sheets from an Excel file and returns a dictionary of DataFrames.

  Args:
    filepath: The path to the Excel file.

  Returns:
    A dictionary where keys are sheet names and values are pandas DataFrames.
    Returns an empty dictionary if the file does not exist or if an error occurs.
  """
  try:
    excel_file = pd.ExcelFile(filepath)
    all_sheets = {}
    for sheet_name in excel_file.sheet_names:
      try:
        all_sheets[sheet_name] = pd.read_excel(excel_file, sheet_name=sheet_name)
      except Exception as e:
        print(f"Error reading sheet '{sheet_name}': {e}")
        # Handle the error as needed (e.g., skip the sheet, return an empty DataFrame, etc.)
    return all_sheets
  except FileNotFoundError:
    print(f"File not found: {filepath}")
    return {}
  except Exception as e:
    print(f"An error occurred: {e}")
    return {}

# Example usage:
filepath = "/content/Intelligent Sourcing.xlsx"
all_sheets_data = read_all_sheets(filepath)

# Access data from individual sheets:
if all_sheets_data:
    for sheet_name, df in all_sheets_data.items():
        print(f"Sheet: {sheet_name}")
        print(df.head()) # Print first few rows for preview
        print("-" * 20)


In [ ]:
import pandas as pd

# Assuming the file is in the /content directory
try:
  df = pd.read_excel('/content/Intelligent Sourcing.xlsx', sheet_name='Weightage')
  print(df)
except FileNotFoundError:
  print("Error: File not found at /content/Intelligent Sourcing.xlsx")
except Exception as e:
  print(f"An error occurred: {e}")


   Variable  Weightage
0      Cost        1.0
1  Priority        0.8
2  Distance        0.6
3      Days        0.4


In [ ]:
Weightage_Cost = 1
Weightage_Priority = 0.8
Weightage_distance = 0.6
Weightage_days = 0.4

# Set number of Warehouses, Products and Orders

In [97]:
num_of_warehouses = 4
num_of_products = 4
num_of_orders = 2

## Warehouse Data

In [98]:
# Read Warehouse data CSV
stock_data_csv = "Warehouse"
for p in range(1, num_of_products+1):
    stock_data_csv += f',Product#{p}'
stock_data_csv += '\n'

for w in range(1, num_of_warehouses+1):
    stock_data_csv += f"Warehouse#{w}"
    for p in range(1, num_of_products+1):
        stock_data_csv += f',{rng.integers(1, 1000)}'
    stock_data_csv += '\n'

stock_data = pd.read_csv(StringIO(stock_data_csv))
stock_data

,Warehouse,Product#1,Product#2,Product#3,Product#4
0,Warehouse#1,453,173,690,113
1,Warehouse#2,278,414,674,395
2,Warehouse#3,879,125,971,555
3,Warehouse#4,1,326,504,367


In [99]:
# Creates a dictionary of the available stock of each product in each warehouse

Warehouses = stock_data['Warehouse'].to_list()
Products = stock_data.columns[1:]

stock = makeDict([Warehouses, Products], stock_data.drop('Warehouse', axis=1).values, default=0)

## Order Data

In [ ]:
# Read Order data CSV
order_data_csv = "Order"
for p in range(1, num_of_products+1):
    order_data_csv += f',Product#{p}'
order_data_csv += '\n'

for o in range(1, num_of_orders+1):
    order_data_csv += f"Order#{o}"
    for p in range(1, num_of_products+1):
        order_data_csv += f',{rng.integers(1, 10)}'
    order_data_csv += '\n'

order_data = pd.read_csv(StringIO(order_data_csv))
order_data

,Order,Product#1,Product#2,Product#3,Product#4
0,Order#1,2,7,3,5
1,Order#2,1,5,7,6


In [101]:
# Creates a dictionary of the number of each product in each deamnd

Orders = order_data['Order'].to_list()

quantity = makeDict([Orders, Products], order_data.drop('Order', axis=1).values, default=0)

## Cost / Distance / Days Details

In [ ]:
# Cost Distance Days Data
cost_data_csv = "Warehouse,Order,Product,Cost\n"
distance_data_csv = "Warehouse,Order,Product,Distance\n"
days_data_csv = "Warehouse,Order,Product,Days\n"

for w in Warehouses:
  for o in Orders:
    for p in Products:
      cost_data_csv += f"{w},{o},{p},{rng.integers(1, 1000)}\n"
      distance_data_csv += f"{w},{o},{p},{rng.integers(1, 100)}\n"
      days_data_csv += f"{w},{o},{p},{rng.integers(1, 30)}\n"

cost_data = pd.read_csv(StringIO(cost_data_csv))
distance_data = pd.read_csv(StringIO(distance_data_csv))
days_data = pd.read_csv(StringIO(days_data_csv))

In [103]:
cost_data

,Warehouse,Order,Product,Cost
0,Warehouse#1,Order#1,Product#1,363
1,Warehouse#1,Order#1,Product#2,229
2,Warehouse#1,Order#1,Product#3,610
3,Warehouse#1,Order#1,Product#4,502
4,Warehouse#1,Order#2,Product#1,349
5,Warehouse#1,Order#2,Product#2,222
6,Warehouse#1,Order#2,Product#3,532
7,Warehouse#1,Order#2,Product#4,748
8,Warehouse#2,Order#1,Product#1,289
9,Warehouse#2,Order#1,Product#2,255


In [104]:
distance_data

,Warehouse,Order,Product,Distance
0,Warehouse#1,Order#1,Product#1,45
1,Warehouse#1,Order#1,Product#2,3
2,Warehouse#1,Order#1,Product#3,84
3,Warehouse#1,Order#1,Product#4,6
4,Warehouse#1,Order#2,Product#1,28
5,Warehouse#1,Order#2,Product#2,22
6,Warehouse#1,Order#2,Product#3,80
7,Warehouse#1,Order#2,Product#4,9
8,Warehouse#2,Order#1,Product#1,47
9,Warehouse#2,Order#1,Product#2,52


In [105]:
days_data

,Warehouse,Order,Product,Days
0,Warehouse#1,Order#1,Product#1,28
1,Warehouse#1,Order#1,Product#2,12
2,Warehouse#1,Order#1,Product#3,25
3,Warehouse#1,Order#1,Product#4,29
4,Warehouse#1,Order#2,Product#1,26
5,Warehouse#1,Order#2,Product#2,12
6,Warehouse#1,Order#2,Product#3,13
7,Warehouse#1,Order#2,Product#4,26
8,Warehouse#2,Order#1,Product#1,8
9,Warehouse#2,Order#1,Product#2,16


In [106]:
# Calculate the minimum and maximum cost
min = cost_data['Cost'].min()
max = cost_data['Cost'].max()

# Normalize the 'Cost' column
cost_data['Cost'] = (cost_data['Cost'] - min) / (max - min)

In [107]:
# Creates a dictionary of the cost for each Warehouse, Order and Product Combination

# Reshape to match the dimensions of Warehouses, Orders, Products
cost_values = cost_data["Cost"].values.reshape(len(Warehouses), len(Orders), len(Products))

cost = makeDict([Warehouses, Orders, Products], cost_values, default=0)

In [108]:
# Calculate the minimum and maximum distance
min = distance_data['Distance'].min()
max = distance_data['Distance'].max()

# Normalize the 'Cost' column
distance_data['Distance'] = (distance_data['Distance'] - min) / (max - min)

In [109]:
# Creates a dictionary of the distance for each Warehouse, Order and Product Combination

# Reshape to match the dimensions of Warehouses, Orders, Products
distance_values = distance_data["Distance"].values.reshape(len(Warehouses), len(Orders), len(Products))

distance = makeDict([Warehouses, Orders, Products], distance_values, default=0)

In [110]:
# Calculate the minimum and maximum days
min = days_data['Days'].min()
max = days_data['Days'].max()

# Normalize the 'Cost' column
days_data['Days'] = (days_data['Days'] - min) / (max - min)

In [111]:
# Creates a dictionary of the days for each Warehouse, Order and Product Combination

# Reshape to match the dimensions of Warehouses, Orders, Products
days_values = days_data["Days"].values.reshape(len(Warehouses), len(Orders), len(Products))

Days = makeDict([Warehouses, Orders, Products], days_values, default=0)

## Priority

In [112]:
# Read Priority data CSV
priority_data_csv = """
Warehouse,Priority
Warehouse#1,100
Warehouse#2,200
Warehouse#3,300
Warehouse#4,1000
"""

priority_data = pd.read_csv(StringIO(priority_data_csv))
priority_data

,Warehouse,Priority
0,Warehouse#1,100
1,Warehouse#2,200
2,Warehouse#3,300
3,Warehouse#4,1000


In [113]:
# Calculate the minimum and maximum cost
min = priority_data['Priority'].min()
max = priority_data['Priority'].max()

# Normalize the 'Cost' column
priority_data['Priority'] = (priority_data['Priority'] - min) / (max - min)

In [114]:
# Creates a dictionary of the priority for warehouse

priority = makeDict([Warehouses], priority_data.drop('Warehouse', axis=1).values.reshape(len(Warehouses)), default=0)
priority

defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
            {'Warehouse#1': 0.0,
             'Warehouse#2': 0.1111111111111111,
             'Warehouse#3': 0.2222222222222222,
             'Warehouse#4': 1.0})

# Linear Optimization

## Define Variables - Routes

In [115]:
# Creates a list of tuples containing all the possible routes for transport
routes = [(w, o, s) for w in Warehouses for o in Orders for s in Products]

In [116]:
# A dictionary called 'Vars' is created to contain the referenced variables(the routes)
variable = LpVariable.dicts("Route", (Warehouses, Orders, Products), 0, None, LpInteger)

'''
for w in Warehouses:
  for o in Orders:
    for p in Products:
      print(f'The variable for transporting {p} from {w} to {o} is {variable[w][o][p]}')
'''

"\nfor w in Warehouses:\n  for o in Orders:\n    for p in Products:\n      print(f'The variable for transporting {p} from {w} to {o} is {variable[w][o][p]}')\n"

## Create cost function

In [ ]:
# Creates the 'prob' variable to contain the problem data
prob = LpProblem("Sourcing_Problem", LpMinimize)

In [ ]:
# The objective function is added to 'prob' first
prob += (
          lpSum(  [ variable[w][o][p]
                    * (
                        (Weightage_Cost * cost[w][o][p])
                         +
                        (Weightage_Priority *-priority[w])
                         +
                        (Weitage_distance * distance[w][o][p])
                         +
                        (Weitage_days * Days[w][o][p])
                      )
                    for (w,o,p) in routes
                  ]
                ),
          "Sum_of_Costs"
        )

#prob

## Stock Constraints

In [ ]:
# The stock constraints are added to prob for each Warehouse

for w in Warehouses:
  for p in Products:
    prob += (
        lpSum([variable[w][o][p] for o in Orders ]) <= stock[w][p],
        f"Sum_of_{p}_out_of_{w}"
    )

## Order Constraints

In [ ]:
# Quantity constrainsts are added for each Order

for o in Orders:
  for p in Products:
    prob += (
        lpSum([variable[w][o][p] for w in Warehouses ]) == quantity[o][p],
        f"Sum_of_{p}_sent_to_{o}"
    )

## Print Optimization Problem

In [ ]:
prob.name

'Sourcing_Problem'

In [ ]:
prob.numVariables()

64

In [ ]:
prob.numConstraints()

32

In [ ]:
prob.objective

0.7334138918725385*Route_Warehouse#1_Order#1_Product#1 + 0.9936403508771929*Route_Warehouse#1_Order#1_Product#2 + 0.5004564983888292*Route_Warehouse#1_Order#1_Product#3 + 0.7348147153598281*Route_Warehouse#1_Order#1_Product#4 + 1.4641962047977086*Route_Warehouse#1_Order#2_Product#1 + 1.1231023988542785*Route_Warehouse#1_Order#2_Product#2 + 1.301624597207304*Route_Warehouse#1_Order#2_Product#3 + 0.8007116004296455*Route_Warehouse#1_Order#2_Product#4 + 1.3114974937343358*Route_Warehouse#1_Order#3_Product#1 + 1.3095148585750092*Route_Warehouse#1_Order#3_Product#2 + 0.5700635517364842*Route_Warehouse#1_Order#3_Product#3 + 1.0677721088435375*Route_Warehouse#1_Order#3_Product#4 + 1.4632742570712496*Route_Warehouse#1_Order#4_Product#1 + 1.0032223415682062*Route_Warehouse#1_Order#4_Product#2 + 0.9083959899749373*Route_Warehouse#1_Order#4_Product#3 + 1.602407805227354*Route_Warehouse#1_Order#4_Product#4 + 1.3525271512113617*Route_Warehouse#2_Order#1_Product#1 + 0.4433300513187731*Route_Warehous

In [ ]:
prob.constraints

OrderedDict([('Sum_of_Product#1_out_of_Warehouse#1',
              1*Route_Warehouse#1_Order#1_Product#1 + 1*Route_Warehouse#1_Order#2_Product#1 + 1*Route_Warehouse#1_Order#3_Product#1 + 1*Route_Warehouse#1_Order#4_Product#1 + -5.0 <= 0),
             ('Sum_of_Product#2_out_of_Warehouse#1',
              1*Route_Warehouse#1_Order#1_Product#2 + 1*Route_Warehouse#1_Order#2_Product#2 + 1*Route_Warehouse#1_Order#3_Product#2 + 1*Route_Warehouse#1_Order#4_Product#2 + -1.0 <= 0),
             ('Sum_of_Product#3_out_of_Warehouse#1',
              1*Route_Warehouse#1_Order#1_Product#3 + 1*Route_Warehouse#1_Order#2_Product#3 + 1*Route_Warehouse#1_Order#3_Product#3 + 1*Route_Warehouse#1_Order#4_Product#3 + -5.0 <= 0),
             ('Sum_of_Product#4_out_of_Warehouse#1',
              1*Route_Warehouse#1_Order#1_Product#4 + 1*Route_Warehouse#1_Order#2_Product#4 + 1*Route_Warehouse#1_Order#3_Product#4 + 1*Route_Warehouse#1_Order#4_Product#4 + -1.0 <= 0),
             ('Sum_of_Product#1_out_of_Wareh

In [ ]:
#Check for duplicate variable names
prob.checkDuplicateVars()

#Check for unused constraint names
prob.unusedConstraintName()

'_C1'

## Solve

In [ ]:
# The problem data is written to an .lp file
#prob.writeLP("DistributionProblem.lp")

# The problem is solved using PuLP's choice of Solver
prob.solve()

# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])


Status: Optimal


## Detailed Report

In [ ]:
## Detailed Report
print("=================================")
print("        Detailed Report:")
print("=================================")

print("\n       Setup Report:")
print("=================================")
print('Warehouses:')
for w in Warehouses:
    print(w)
print('\nOrders:')
for o in Orders:
    print(o)
print('\nProducts:')
for p in Products:
    print(p)



print("\n      Inventory Report:")
print("=================================")
for w in Warehouses:
    print(f"Warehouse {w}:")
    for p in Products:
        print(f"  Product {p}: {stock[w][p]} units")

print("\n       Demand Report:")
print("=================================")
for o in Orders:
    print(f"Order {o}:")
    for p in Products:
        print(f"  Product {p}: {quantity[o][p]} units")

print("\n       Cost Report:")
print("=================================")
for w in Warehouses:
    print(f"Warehouse {w}:")
    for o in Orders:
        print(f"  Order {o}:")
        for p in Products:
            print(f"    Product {p}: {cost[w][o][p]}")

print("\n       Priority Report:")
print("=================================")
for w in Warehouses:
    print(f"Warehouse {w}:")
    print(f"  Priority: {priority[w]}")



print("\n   Fulfillment Report:")
print("========================")

print("\nSupply from each Warehouse to each Order:")
for w in Warehouses:
    print(f"Warehouse {w}:")
    for p in Products:
        for o in Orders:
            route_var = variable[w][o][p]
            if route_var.varValue > 0:
                print(f"  supplies {route_var.varValue} units of {p}")
                print(f"    to Order {o}")

print("\nUnfulfilled Demand:")
unfulfilled_flag = False
for o in Orders:
    for p in Products:
        total_received = sum(variable[w][o][p].varValue for w in Warehouses)
        unfulfilled = quantity[o][p] - total_received
        if unfulfilled > 0:
            print(f"Order {o}: Unfulfilled demand for {p} : {unfulfilled}")
if unfulfilled_flag is False:
    print("All orders are fulfilled")



print("\n Validation Report:")
print("========================")

print("\nValidation - Warehouse level:")
for w in Warehouses:
  print(f"Warehouse {w}:")
  for p in Products:
    total_supplied = sum(variable[w][o][p].varValue for o in Orders)

    if total_supplied > stock[w][p]:
      print(f"ERROR!!! - Warehouse {w}:")
      print(f"  Product {p}: Shipped ({total_supplied} units) is > than Stock ({stock[w][p]} units)")

    print(f"  Product {p}: Shipped ({total_supplied} units) is <= than Stock ({stock[w][p]} units). Remaining is {stock[w][p] - total_supplied}")

print("\nValidation - Order level:")
for o in Orders:
  print(f"Order {w}:")
  for p in Products:
    total_received= sum(variable[w][o][p].varValue for w in Warehouses)

    if total_received != quantity[o][p]:
      print(f"ERROR!!! - Order {o}:")
      print(f"  Product {p}: Received ({total_received} units) is != than Quantity ({quantity[o][p]} units)")

    print(f"  Product {p}: Received ({total_received} units) is = than Quantity ({quantity[o][p]} units)")


#print(f"\nTotal Cost: {value(prob.objective)}")

        Detailed Report:

       Setup Report:
Warehouses:
Warehouse#1
Warehouse#2
Warehouse#3
Warehouse#4

Orders:
Order#1
Order#2
Order#3
Order#4

Products:
Product#1
Product#2
Product#3
Product#4

      Inventory Report:
Warehouse Warehouse#1:
  Product Product#1: 5 units
  Product Product#2: 1 units
  Product Product#3: 5 units
  Product Product#4: 1 units
Warehouse Warehouse#2:
  Product Product#1: 5 units
  Product Product#2: 5 units
  Product Product#3: 1 units
  Product Product#4: 15 units
Warehouse Warehouse#3:
  Product Product#1: 2 units
  Product Product#2: 5 units
  Product Product#3: 10 units
  Product Product#4: 0 units
Warehouse Warehouse#4:
  Product Product#1: 4 units
  Product Product#2: 5 units
  Product Product#3: 0 units
  Product Product#4: 2 units

       Demand Report:
Order Order#1:
  Product Product#1: 2 units
  Product Product#2: 3 units
  Product Product#3: 2 units
  Product Product#4: 3 units
Order Order#2:
  Product Product#1: 4 units
  Product Product#2:

# Fulfillment Solution

In [ ]:
fulfillment_solution = []

for w in Warehouses:
    for p in Products:
        for o in Orders:
            fulfillment_solution.append({
                "Warehouse": w,
                "Product": p,
                "Order": o,
                "Supply Quantity": variable[w][o][p].varValue
            })

fulfillment_solution = pd.DataFrame(fulfillment_solution)
fulfillment_solution

,Warehouse,Product,Order,Supply Quantity
0,Warehouse#1,Product#1,Order#1,2.0
1,Warehouse#1,Product#1,Order#2,0.0
2,Warehouse#1,Product#1,Order#3,0.0
3,Warehouse#1,Product#1,Order#4,0.0
4,Warehouse#1,Product#2,Order#1,0.0
...,...,...,...,...
59,Warehouse#4,Product#3,Order#4,0.0
60,Warehouse#4,Product#4,Order#1,0.0
61,Warehouse#4,Product#4,Order#2,0.0
62,Warehouse#4,Product#4,Order#3,0.0


# Stock Status

In [ ]:
warehouse_stock_status = []

for w in Warehouses:
    for p in Products:
        warehouse_stock_status.append({
            "Warehouse": w,
            "Product": p,
            "Initial Stock": stock[w][p],
            "Supplied Stock": sum(variable[w][o][p].varValue for o in Orders),
            "Remaining Stock": stock[w][p] - sum(variable[w][o][p].varValue for o in Orders)
        })

warehouse_stock_status = pd.DataFrame(warehouse_stock_status)
warehouse_stock_status

,Warehouse,Product,Initial Stock,Supplied Stock,Remaining Stock
0,Warehouse#1,Product#1,5,2.0,3.0
1,Warehouse#1,Product#2,1,0.0,1.0
2,Warehouse#1,Product#3,5,3.0,2.0
3,Warehouse#1,Product#4,1,1.0,0.0
4,Warehouse#2,Product#1,5,5.0,0.0
5,Warehouse#2,Product#2,5,1.0,4.0
6,Warehouse#2,Product#3,1,0.0,1.0
7,Warehouse#2,Product#4,15,6.0,9.0
8,Warehouse#3,Product#1,2,2.0,0.0
9,Warehouse#3,Product#2,5,3.0,2.0


# Write out all data to file

In [ ]:
# Create a DataFrame from the weightage variables
weightage_data = {
    'Variable': ['Cost', 'Priority', 'Distance', 'Days'],
    'Weightage': [Weightage_Cost, Weightage_Priority, Weightage_distance, Weightage_days]
}
weightage_df = pd.DataFrame(weightage_data)

# Display the DataFrame
weightage_df


,Variable,Weightage
0,Cost,1.0
1,Priority,0.8
2,Distance,0.6
3,Days,0.4


In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('Intelligent Sourcing.xlsx', engine='xlsxwriter')

# Write each DataFrame to a different worksheet.
weightage_df.to_excel(writer, sheet_name='Weightage', index=False)
stock_data.to_excel(writer, sheet_name='Stock Data', index=False)
order_data.to_excel(writer, sheet_name='Order Data', index=False)
cost_data.to_excel(writer, sheet_name='Cost Data', index=False)
distance_data.to_excel(writer, sheet_name='Distance Data', index=False)
days_data.to_excel(writer, sheet_name='Days Data', index=False)
priority_data.to_excel(writer, sheet_name='Priority Data', index=False)
fulfillment_solution.to_excel(writer, sheet_name='Fulfillment Solution', index=False)
warehouse_stock_status.to_excel(writer, sheet_name='Warehouse Stock Status', index=False)

# Close the Pandas Excel writer and output the Excel file.
writer.close()

# Gradio

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
import pandas as pd
from pulp import *
from io import StringIO
import numpy as np
import xlsxwriter

# Function to process the inputs and execute the optimization
def process_data(stock_data, order_data, cost_data, priority_data):
    try:
        # Convert inputs to DataFrames
        stock_data = pd.DataFrame(stock_data, columns=["Warehouse", "Product#1", "Product#2", "Product#3", "Product#4"])
        order_data = pd.DataFrame(order_data, columns=["Order", "Product#1", "Product#2", "Product#3", "Product#4"])
        cost_data = pd.DataFrame(cost_data, columns=["Warehouse", "Order", "Product", "Cost"])
        priority_data = pd.DataFrame(priority_data, columns=["Warehouse", "Priority"])

        # Convert numeric columns to appropriate types
        for col in stock_data.columns[1:]:
            stock_data[col] = pd.to_numeric(stock_data[col], errors='coerce')

        for col in order_data.columns[1:]:
            order_data[col] = pd.to_numeric(order_data[col], errors='coerce')

        cost_data["Priority"] = pd.to_numeric(cost_data["Cost"], errors='coerce')

        priority_data["Priority"] = pd.to_numeric(priority_data["Priority"], errors='coerce')

        # Drop rows with invalid data
        #stock_data.dropna(inplace=True)
        #order_data.dropna(inplace=True)
        #priority_data.dropna(inplace=True)
        ##############################################################################################################

        # Creates a dictionary of the available stock of each product in each warehouse

        Warehouses = stock_data['Warehouse'].to_list()
        Products = stock_data.columns[1:]

        stock = makeDict([Warehouses, Products], stock_data.drop('Warehouse', axis=1).values, default=0)

        # Creates a dictionary of the number of each product in each deamnd

        Orders = order_data['Order'].to_list()

        quantity = makeDict([Orders, Products], order_data.drop('Order', axis=1).values, default=0)

        # Calculate the minimum and maximum cost
        min_cost = cost_data['Cost'].min()
        max_cost = cost_data['Cost'].max()

        # Normalize the 'Cost' column
        cost_data['Cost'] = (cost_data['Cost'] - min_cost) / (max_cost - min_cost)

        # Creates a dictionary of the cost for each Warehouse, Order and Product Combination

        # Reshape cost_data["Cost"].values to match the dimensions of Warehouses, Orders, Products
        cost_values = cost_data["Cost"].values.reshape(len(Warehouses), len(Orders), len(Products))
        cost_values.shape

        cost = makeDict([Warehouses, Orders, Products], cost_values, default=0)

        # Calculate the minimum and maximum cost
        min_cost = priority_data['Priority'].min()
        max_cost = priority_data['Priority'].max()

        # Normalize the 'Cost' column
        priority_data['Priority'] = (priority_data['Priority'] - min_cost) / (max_cost - min_cost)

        # Creates a dictionary of the priority for warehouse

        priority = makeDict([Warehouses], priority_data.drop('Warehouse', axis=1).values.reshape(len(Warehouses)), default=0)
        priority

        # Creates a list of tuples containing all the possible routes for transport
        routes = [(w, o, s) for w in Warehouses for o in Orders for s in Products]

        # A dictionary called 'Vars' is created to contain the referenced variables(the routes)
        variable = LpVariable.dicts("Route", (Warehouses, Orders, Products), 0, None, LpInteger)

        for w in Warehouses:
          for o in Orders:
            for p in Products:
              print(f'The variable for transporting {p} from {w} to {o} is {variable[w][o][p]}')

        # Creates the 'prob' variable to contain the problem data
        prob = LpProblem("Distribution Problem", LpMinimize)

        # The objective function is added to 'prob' first
        prob += (
            lpSum([cost[w][o][p] * variable[w][o][p] * -priority[w] for (w,o,p) in routes]),
            "Sum_of_Costs"
        )

        # The stock constraints are added to prob for each Warehouse

        for w in Warehouses:
          for p in Products:
            prob += (
                lpSum([variable[w][o][p] for o in Orders ]) <= stock[w][p],
                f"Sum_of_{p}_out_of_{w}"
            )

        # Quantity constrainsts are added for each Order

        for o in Orders:
          for p in Products:
            prob += (
                lpSum([variable[w][o][p] for w in Warehouses ]) == quantity[o][p],
                f"Sum_of_{p}_sent_to_{o}"
            )

        # The problem is solved using PuLP's choice of Solver
        prob.solve()

        # Capture output
        output = StringIO()
        print("Status:", LpStatus[prob.status], file=output)
        if LpStatus[prob.status] != "Optimal":
            print("Optimization was not successful!!", file=output)
            print("Optimization was not successful!!", file=output)
            print("Optimization was not successful!!", file=output)
            print("Optimization was not successful!!", file=output)
            print("Optimization was not successful!!", file=output)
            print("Optimization was not successful!!", file=output)
            print("Optimization was not successful!!", file=output)
            raise ValueError(f"ERR!! No solution found!! - Status is {LpStatus[prob.status]}")
            return output.getvalue()

        gr.Info(f"Solution found!! - Status is {LpStatus[prob.status]}")


        print("\n   Fulfillment Report:", file=output)
        print("========================", file=output)

        print("\nSupply from each Warehouse to each Order:", file=output)
        for w in Warehouses:
            print(f"Warehouse {w}:", file=output)
            for p in Products:
                for o in Orders:
                    route_var = variable[w][o][p]
                    if route_var.varValue > 0:
                        print(f"  supplies {route_var.varValue} units of {p}", file=output)
                        print(f"    to Order {o}", file=output)

        print("\nUnfulfilled Demand:", file=output)
        unfulfilled_flag = False
        for o in Orders:
            for p in Products:
                total_received = sum(variable[w][o][p].varValue for w in Warehouses)
                unfulfilled = quantity[o][p] - total_received
                if unfulfilled > 0:
                    print(f"Order {o}: Unfulfilled demand for {p} : {unfulfilled}", file=output)
        if unfulfilled_flag is False:
            print("All orders are fulfilled", file=output)


        print("\n Validation Report:", file=output)
        print("========================", file=output)

        print("\nValidation - Warehouse level:", file=output)
        for w in Warehouses:
            print(f"Warehouse {w}:", file=output)
            for p in Products:
                total_supplied = sum(variable[w][o][p].varValue for o in Orders)

                if total_supplied > stock[w][p]:
                    print(f"ERROR!!! - Warehouse {w}:", file=output)
                    print(f"  Product {p}: Shipped ({total_supplied} units) is > than Stock ({stock[w][p]} units)", file=output)

                print(f"  Product {p}: Shipped ({total_supplied} units) is <= than Stock ({stock[w][p]} units). Remaining is {stock[w][p]} - {total_supplied}", file=output)

        print("\nValidation - Order level:", file=output)
        for o in Orders:
            print(f"Order {w}:", file=output)
            for p in Products:
                total_received = sum(variable[w][o][p].varValue for w in Warehouses)

                if total_received != quantity[o][p]:
                    print(f"ERROR!!! - Order {o}:", file=output)
                    print(f"  Product {p}: Received ({total_received} units) is != than Quantity ({quantity[o][p]} units)", file=output)

                print(f"  Product {p}: Received ({total_received} units) is = than Quantity ({quantity[o][p]} units)", file=output)


        # Create a Pandas Excel writer using XlsxWriter as the engine.
        writer = pd.ExcelWriter('/tmp/Intelligent Sourcing.xlsx', engine='xlsxwriter')

        # Write each DataFrame to a different worksheet.
        stock_data.to_excel(writer, sheet_name='Stock Data', index=False)
        order_data.to_excel(writer, sheet_name='Order Data', index=False)
        cost_data.to_excel(writer, sheet_name='Cost Data', index=False)
        priority_data.to_excel(writer, sheet_name='Priority Data', index=False)

        fulfillment_solution = []
        for w in Warehouses:
            for p in Products:
                for o in Orders:
                    fulfillment_solution.append({
                        "Warehouse": w,
                        "Product": p,
                        "Order": o,
                        "Supply Quantity": variable[w][o][p].varValue
                    })

        fulfillment_solution = pd.DataFrame(fulfillment_solution)
        fulfillment_solution.to_excel(writer, sheet_name='Fulfillment Solution', index=False)

        # Close the Pandas Excel writer and output the Excel file.
        writer.close()


        return output.getvalue(), '/tmp/Intelligent Sourcing.xlsx'

    except Exception as e:
        return f"Error: {str(e)}"

# Define example data
stock_example = [["Warehouse#1", 5, 1, 5, 1],
                 ["Warehouse#2", 5, 5, 1, 15],
                 ["Warehouse#3", 2, 5, 10, 0],
                 ["Warehouse#4", 4, 5, 0, 2]]
order_example = [["Order#1", 2, 3, 2, 3],
                 ["Order#2", 4, 1, 4, 1],
                 ["Order#3", 5, 3, 5, 3],
                 ["Order#4", 2, 2, 2, 2]]
cost_example = [
    ["Warehouse#1", "Order#1", "Product#1", 103],
    ["Warehouse#1", "Order#1", "Product#2", 436],
    ["Warehouse#1", "Order#1", "Product#3", 861],
    ["Warehouse#1", "Order#1", "Product#4", 271],
    ["Warehouse#1", "Order#2", "Product#1", 107],
    ["Warehouse#1", "Order#2", "Product#2", 72],
    ["Warehouse#1", "Order#2", "Product#3", 701],
    ["Warehouse#1", "Order#2", "Product#4", 21],
    ["Warehouse#1", "Order#3", "Product#1", 615],
    ["Warehouse#1", "Order#3", "Product#2", 122],
    ["Warehouse#1", "Order#3", "Product#3", 467],
    ["Warehouse#1", "Order#3", "Product#4", 215],
    ["Warehouse#1", "Order#4", "Product#1", 331],
    ["Warehouse#1", "Order#4", "Product#2", 459],
    ["Warehouse#1", "Order#4", "Product#3", 88],
    ["Warehouse#1", "Order#4", "Product#4", 373],
    ["Warehouse#2", "Order#1", "Product#1", 100],
    ["Warehouse#2", "Order#1", "Product#2", 872],
    ["Warehouse#2", "Order#1", "Product#3", 664],
    ["Warehouse#2", "Order#1", "Product#4", 131],
    ["Warehouse#2", "Order#2", "Product#1", 662],
    ["Warehouse#2", "Order#2", "Product#2", 309],
    ["Warehouse#2", "Order#2", "Product#3", 770],
    ["Warehouse#2", "Order#2", "Product#4", 344],
    ["Warehouse#2", "Order#3", "Product#1", 492],
    ["Warehouse#2", "Order#3", "Product#2", 414],
    ["Warehouse#2", "Order#3", "Product#3", 806],
    ["Warehouse#2", "Order#3", "Product#4", 386],
    ["Warehouse#2", "Order#4", "Product#1", 192],
    ["Warehouse#2", "Order#4", "Product#2", 956],
    ["Warehouse#2", "Order#4", "Product#3", 277],
    ["Warehouse#2", "Order#4", "Product#4", 161],
    ["Warehouse#3", "Order#1", "Product#1", 460],
    ["Warehouse#3", "Order#1", "Product#2", 314],
    ["Warehouse#3", "Order#1", "Product#3", 22],
    ["Warehouse#3", "Order#1", "Product#4", 253],
    ["Warehouse#3", "Order#2", "Product#1", 748],
    ["Warehouse#3", "Order#2", "Product#2", 857],
    ["Warehouse#3", "Order#2", "Product#3", 561],
    ["Warehouse#3", "Order#2", "Product#4", 475],
    ["Warehouse#3", "Order#3", "Product#1", 59],
    ["Warehouse#3", "Order#3", "Product#2", 511],
    ["Warehouse#3", "Order#3", "Product#3", 682],
    ["Warehouse#3", "Order#3", "Product#4", 476],
    ["Warehouse#3", "Order#4", "Product#1", 700],
    ["Warehouse#3", "Order#4", "Product#2", 976],
    ["Warehouse#3", "Order#4", "Product#3", 783],
    ["Warehouse#3", "Order#4", "Product#4", 190],
    ["Warehouse#4", "Order#1", "Product#1", 958],
    ["Warehouse#4", "Order#1", "Product#2", 687],
    ["Warehouse#4", "Order#1", "Product#3", 958],
    ["Warehouse#4", "Order#1", "Product#4", 563],
    ["Warehouse#4", "Order#2", "Product#1", 876],
    ["Warehouse#4", "Order#2", "Product#2", 567],
    ["Warehouse#4", "Order#2", "Product#3", 244],
    ["Warehouse#4", "Order#2", "Product#4", 832],
    ["Warehouse#4", "Order#3", "Product#1", 505],
    ["Warehouse#4", "Order#3", "Product#2", 131],
    ["Warehouse#4", "Order#3", "Product#3", 485],
    ["Warehouse#4", "Order#3", "Product#4", 819],
    ["Warehouse#4", "Order#4", "Product#1", 647],
    ["Warehouse#4", "Order#4", "Product#2", 21],
    ["Warehouse#4", "Order#4", "Product#3", 841],
    ["Warehouse#4", "Order#4", "Product#4", 167],
]
priority_example = [["Warehouse#1", 100],
                    ["Warehouse#2", 200],
                    ["Warehouse#3", 300],
                    ["Warehouse#4", 1000]]

# Create Gradio interface
with gr.Blocks(gr.themes.Soft()) as app:
    gr.Markdown("# Intelligent Sourcing Optimization App")
    gr.Markdown("## Created by Krishna Kumar.S")
    gr.Markdown("### PoC - Work in Progress!!")

    with gr.Tab("Stock Data"):
        stock_data_input = gr.Dataframe(label="Stock Data", headers=["Warehouse", "Product#1", "Product#2", "Product#3", "Product#4"],
                                        value=stock_example, datatype="str")
    with gr.Tab("Order Data"):
        order_data_input = gr.Dataframe(label="Order Data", headers=["Order", "Product#1", "Product#2", "Product#3", "Product#4"],
                                        value=order_example, datatype="str")
    with gr.Tab("Cost Data"):
        cost_data_input = gr.Dataframe(label="Cost Data", headers=["Warehouse", "Order", "Product", "Cost"],
                                        value=cost_example, datatype="str")
    with gr.Tab("Priority Data"):
        priority_data_input = gr.Dataframe(label="Priority Data", headers=["Warehouse", "Priority"],
                                           value=priority_example, datatype="str")

    file_path = gr.State()  # State to hold the file path dynamically

    submit_button = gr.Button("Run Optimization")
    download_button = gr.File(label="Download Excel", file_types=[".xlsx"])
    output_text = gr.Textbox(label="Output", lines=110, interactive=False)

    submit_button.click(process_data, inputs=[stock_data_input, order_data_input, cost_data_input, priority_data_input], outputs=[output_text, file_path])

    # Update the download button dynamically with the generated file path
    file_path.change(
        fn=lambda path, : path,
        inputs=file_path,
        outputs=download_button
    )

app.launch()
